In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, TensorDataset, DataLoader
import pandas as pd
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import os
target_path = '/your_path'
for dirname, _, filenames in os.walk(target_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [41]:
# Check device, nếu GPU sẵn sàng thì sẽ dùng, không thì dùng CPU

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Switching to CPU.")

GPU is available.


In [42]:
# Load data lên

load_dataset  = load_dataset("hungnm/vietnamese-medical-qa")
tokenizer = get_tokenizer(None, language='vi')
samples = 100

class TextDataset(Dataset):
      def __init__(self, input_x, input_y):
          self.input_x = input_x
          self.input_y = input_y

      def __len__(self):
            return len(self.input_x+self.input_y)

      def __getitem__(self, idx):
            input_x_i = self.input_x[idx]
            input_y_i = self.input_y[idx]
            return {"text_input": input_x_i, "text_output": input_y_i}

def preprocess(load_dataset):
  question = load_dataset['train']['question'][:samples]
  answer = load_dataset['train']['answer'][:samples]
  return question, answer

# Tạo dataset và tokenize text

dataset = TextDataset(*preprocess(load_dataset))
tokenized_input = []
tokenized_output = []

for text in iter(dataset):
  tokens_text_input = tokenizer(text['text_input'])
  tokenized_input.append(tokens_text_input)

for label in iter(dataset):
  tokens_text_output = tokenizer(text['text_output'])
  tokenized_output.append(tokens_text_output)

tokenized = tokenized_input + tokenized_output

# Tạo vocab

vocab = build_vocab_from_iterator(iter(tokenized))
index_to_word = vocab.get_itos()


# Tính toán chiều dài tối đa của các vector
max_length = max(len(tokens) for tokens in tokenized)


  0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# Định nghĩa các hàm chuyển đổi

def tokenize_input_to_matrix(tokenized_input, vocab_length, max_length):
    batch_size = len(tokenized_input)
    input_matrix = torch.zeros((batch_size, max_length, vocab_length), dtype=torch.long)
    for i, sentence in enumerate(tokenized_input):
        for j, token in enumerate(sentence):
            if j >= max_length:
                break
            input_matrix[i, j, vocab([token])[0]] = vocab([token])[0]
    return torch.tensor(input_matrix, dtype=torch.long)

def tokenize_ques_input_to_matrix(sentence, vocab_length, max_length):
    input_matrix = torch.zeros((1, max_length, vocab_length), dtype=torch.long)
    for i, token in enumerate(sentence):
        if i >= max_length:
            break
        input_matrix[0, i, vocab([token])[0]] = vocab([token])[0]
    return torch.tensor(input_matrix, dtype=torch.long)

def vector_to_text(output_vectors, vocabulary):
    text = []
    # Lấy chỉ mục của từ có xác suất cao nhất
    for output_vector in output_vectors:
        predicted_index = torch.argmax(output_vector[0]).item()
        # Tìm từ tương ứng trong từ điển
        predicted_word = index_to_word[predicted_index]
        text.append(predicted_word)

    return " ".join(text)

In [44]:
# Tạo vector x_input cho mỗi câu trong tập dữ liệu đã được padding
x_input_vectors = tokenize_input_to_matrix(tokenized_input, len(vocab), max_length).to(device)
# Tạo vector y_input cho mỗi câu trong tập dữ liệu đã được padding
y_input_vectors = tokenize_input_to_matrix(tokenized_output, len(vocab), max_length).to(device)

/tmp/ipykernel_34/620528770.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(input_matrix, dtype=torch.long)


In [45]:
# Định nghĩa các siêu tham số và kiến trúc model

vocab_size = len(vocab)  # Số lượng từ trong từ vựng
d_model = 512  # Kích thước vector ẩn cho mỗi từ
num_heads = 8  # Số lượng head trong multi-head attention
num_layers = 6  # Số lượng layer trong encoder
dim_feedforward = 1024  # Kích thước của mạng feedforward trong transformer
max_seq_length = max_length  # Độ dài tối đa của chuỗi đầu vào
dropout = 0.1  # Tỷ lệ dropout

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, input_vocab_size=vocab_size, output_vocab_size=vocab_size, d_model=d_model, num_heads=num_heads, num_encoder_layers=num_layers, num_decoder_layers=num_layers):
        super(TransformerModel, self).__init__()
        self.embedding_encoder = nn.Embedding(input_vocab_size, d_model)
        self.embedding_decoder = nn.Embedding(output_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads)
        decoder_layer = nn.TransformerDecoderLayer(d_model, num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers)

        self.fc = nn.Sequential(nn.Linear(d_model, dim_feedforward),
                                nn.Linear(dim_feedforward, output_vocab_size))

    def forward(self, src, tgt):
        src = self.embedding_encoder(src)
        src = self.pos_encoder(src)

        tgt = self.embedding_decoder(tgt)
        tgt = self.pos_encoder(tgt)

        memory = self.transformer_encoder(src)
        output = self.transformer_decoder(tgt, memory)

        output = self.fc(output)
        return output

# Định nghĩa train loop
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0

    for input_xs, input_ys in dataloader:
        input_xs, input_ys = input_xs.squeeze(0), input_ys.squeeze(0)
        optimizer.zero_grad()
        outputs = model(input_xs, input_ys)
        loss = criterion(outputs, input_ys)
        loss.backward()
        optimizer.step()
        torch.save(model.state_dict(), '/kaggle/working/model_weights.pth')
        total_loss += loss.item()

    return total_loss / len(dataloader)

In [46]:
#Gọi model, loss func và optimizer

learning_rate = 0.1

print(x_input_vectors[0].shape, y_input_vectors[0].shape)

model = TransformerModel().to(device)
# model.load_state_dict(torch.load('/kaggle/input/weight/model_weights.pth'))
model = nn.DataParallel(model)
input_dataset = TensorDataset(x_input_vectors, y_input_vectors)
dataloader = DataLoader(input_dataset, shuffle=True, batch_size=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

torch.Size([153, 1093]) torch.Size([153, 1093])


In [47]:
#Train loop model

num_epochs = 50

for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, dataloader, criterion, optimizer)
    print(f'Epoch [{epoch}/{num_epochs}], Loss: {train_loss:.4f}')

In [48]:
#Test model

input_str = str(input("Đặt câu hỏi đến bác sĩ: "))
ques_tokens = tokenizer(input_str)
ques_input = tokenize_ques_input_to_matrix(ques_tokens, len(vocab), max_length).squeeze()
print(ques_input.shape)
model.eval()
with torch.no_grad():
    output = model(ques_input, (ques_input != 0).long())
    print(vector_to_text(output, vocab))

Đặt câu hỏi đến bác sĩ:  ho và sổ mũi


/tmp/ipykernel_34/620528770.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(input_matrix, dtype=torch.long)


torch.Size([153, 1093])
giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác giác
